# Transpiler plugins

To facilitate the development and reuse of custom transpilation code by the wider community of Qiskit users, Qiskit supports a plugin interface that enables third-party Python packages to declare that they provide extended transpilation functionality accessible via Qiskit.

Currently, third-party plugins can provide extended transpilation functionality in three ways:

- A [transpiler stage plugin](/api/qiskit/transpiler_plugins) provides a pass manager that can be used in place of one of the [6 stages](transpiler-stages) of a preset staged pass manager: `init`, `layout`, `routing`, `translation`, `optimization`, and `scheduling`.
- A [unitary synthesis plugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin) provides extended functionality for unitary gate synthesis.
- A [high-level synthesis plugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin) provides extended functionality for synthesizing "high-level objects" such as linear functions or Clifford operators. High-level objects are represented by subclasses of the [Operation](/api/qiskit/qiskit.circuit.Operation) class.

The rest of the page describes how to list available plugins, install new ones, and use them.

## List available plugins and install new ones

Qiskit already includes some built-in plugins for transpilation. To install more, you can use your Python package manager. For example, you might run `pip install qiskit-toqm` to install the [Qiskit TOQM](https://github.com/qiskit-toqm/qiskit-toqm) routing stage plugin. A number of third-party plugins are part of the [Qiskit ecosystem](https://qiskit.github.io/ecosystem/#transpiler_plugin).

### List available transpiler stage plugins

Use the [list_stage_plugins](/api/qiskit/transpiler_plugins#qiskit.transpiler.preset_passmanagers.plugin.list_stage_plugins) function, passing the name of the stage whose plugins you want to list.

In [1]:
from qiskit.transpiler.preset_passmanagers.plugin import list_stage_plugins

list_stage_plugins("layout")

['default', 'dense', 'sabre', 'trivial']

In [2]:
list_stage_plugins("routing")

['basic', 'lookahead', 'none', 'sabre', 'stochastic']

 If `qiskit-toqm` were installed, then `toqm` would appear in the list of `routing` plugins.

### List available unitary synthesis plugins

Use the [unitary_synthesis_plugin_names](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.unitary_synthesis_plugin_names) function.

In [3]:
from qiskit.transpiler.passes.synthesis import unitary_synthesis_plugin_names

unitary_synthesis_plugin_names()

['aqc', 'default', 'sk']

### List available high-level sythesis plugins

Use the [high_level_synthesis_plugin_names](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.high_level_synthesis_plugin_names) function, passing the name of the type of "high-level object" to be synthesized. The name corresponds to the [`name`](/api/qiskit/qiskit.circuit.Operation#name) attribute of the [Operation](/api/qiskit/qiskit.circuit.Operation) class representing the type of object being synthesized.

In [4]:
from qiskit.transpiler.passes.synthesis import high_level_synthesis_plugin_names

high_level_synthesis_plugin_names("clifford")

['ag', 'bm', 'default', 'greedy', 'layers', 'lnn']

You can use the [HighLevelSynthesisPluginManager](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPluginManager) class to list the names of all high-level synthesis plugins:

In [21]:
from qiskit.transpiler.passes.synthesis.plugin import HighLevelSynthesisPluginManager

HighLevelSynthesisPluginManager().plugins.names()

['clifford.ag',
 'clifford.bm',
 'clifford.default',
 'clifford.greedy',
 'clifford.layers',
 'clifford.lnn',
 'linear_function.default',
 'linear_function.kms',
 'linear_function.pmh',
 'permutation.acg',
 'permutation.basic',
 'permutation.default',
 'permutation.kms',
 'permutation.token_swapper']

## Use a plugin

### Use a transpiler stage plugin

To use a transpiler stage plugin, specify its name with the appropriate argument to [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#generate_preset_pass_manager) or [`transpile`](/api/qiskit/compiler#qiskit.compiler.transpile). The argument is formed by appending `_method` to the name of the transpilation stage. For example, to use the `stochastic` routing plugin, we would specify `stochastic` for the `routing_method` argument:

In [5]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")

pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, routing_method="stochastic"
)

### Use a unitary synthesis plugin

To use a unitary synthesis plugin, specify its name as the `unitary_synthesis_method` argument to [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#generate_preset_pass_manager) or [`transpile`](/api/qiskit/compiler#qiskit.compiler.transpile):

In [6]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3,
    backend=backend,
    unitary_synthesis_method="sk",
    unitary_synthesis_plugin_config=dict(basis_gates=["cz", "id", "rz", "sx", "x"]),
)

Unitary synthesis is used in the `init`, `translation`, and `optimization` stages of the staged pass manager returned by [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#generate_preset_pass_manager) or used in [`transpile`](/api/qiskit/compiler#qiskit.compiler.transpile). See [Transpiler stages](transpiler-stages) for a description of these stages.

Use the `unitary_synthesis_plugin_config` argument, a free-form dictionary, to pass options for the unitary synthesis method. The documentation of the synthesis method should explain the options it supports. See [this list](/api/qiskit/transpiler_synthesis_plugins#unitary-synthesis-plugins-2) for links to the documentation of the built-in unitary synthesis plugins.

### Use a high-level synthesis plugin

First, create an [HLSConfig](/api/qiskit/qiskit.transpiler.passes.HLSConfig) to
store the names of the plugins to use for various high-level objects. For example:

In [7]:
from qiskit.transpiler.passes import HLSConfig

hls_config = HLSConfig(clifford=["layers"], linear_function=["pmh"])

This code cell creates a high-level synthesis configuration that uses the `layers` plugin
for synthesizing [Clifford](/api/qiskit/qiskit.quantum_info.Clifford#clifford) objects and the `pmh` plugin for synthesizing
[LinearFunction](/api/qiskit/qiskit.circuit.library.LinearFunction#linearfunction) objects. The names of the keyword arguments correspond to the
[`name`](/api/qiskit/qiskit.circuit.Operation#name) attribute of the [Operation](/api/qiskit/qiskit.circuit.Operation) class representing the type of object being synthesized.
For each high-level object, the list of given plugins are tried in sequence until one of them
succeeds (in the example above, each list only contains a single plugin).

In addition to specifying
a plugin by its name, you can instead pass a `(name, options)` tuple, where the second element of the tuple is a dictionary containing options for the plugin. The documentation of the synthesis method should explain the options it supports. See [this list](/api/qiskit/transpiler_synthesis_plugins#high-level-synthesis-plugins-2) for links to the documentation of the built-in high-level synthesis plugins.

Once you have created the `HLSConfig` object, pass it as the
`hls_config` argument to [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#generate_preset_pass_manager) or [`transpile`](/api/qiskit/compiler#qiskit.compiler.transpile):

In [8]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, hls_config=hls_config
)

High-level synthesis is used in the `init`, `translation`, and `optimization` stages of the staged pass manager returned by [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#generate_preset_pass_manager) or used in [`transpile`](/api/qiskit/compiler#qiskit.compiler.transpile). See [Transpiler stages](transpiler-stages) for a description of these stages.